In [1]:
import numpy as np
import os
import subprocess 
import MDAnalysis as md
from MDAnalysis.analysis.leaflet import LeafletFinder
import re

# Notebook for constructing inverse hexagonal phase, using 4 channels with 21nt dsPoly A RNA inside each channel
A hydration of 3 water beads per lipid is used. The needed CL ions for neutralization is subtracted from the number of W beads for stability.

The water and ions are added with packmol, so various hydration levels can be used

### Below paths you will need to update yourself!


In the function write_topol, you will maybe also have to check that the paths for the itp files needed is correct for when writing the top file

In [2]:
main_dir = '.'
mdp_loc_BL = 'MDPs/'
ILs_itp = 'MDPs/ILs.itp'
insane = "insane-ILs.py"

PCG_path = '/projects/DAMM/TS2CG1.1-master/PCG' #This is only applicable for TS2CG1.1
packmol = '/projects/DAMM/packmol-20.14.3/packmol'

RNA = 'PDBs/RNA.pdb'
RNA_itp = 'ITPs/RNA.itp'

####  functions

In [3]:
def get_radius (x,y,z):
    # Calculate the centroid of the cylinder (center axis)
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    centroid_z = np.mean(z)

    # Calculate the distances from the centroid to each point
    distances = np.sqrt((x - centroid_x)**2 + (y - centroid_y)**2 + (z - centroid_z)**2)

    # Find the minimum distance, which represents the internal radius of the cylinder
    internal_radius = np.min(distances)

    return internal_radius

def write_packmol_input(channel_radius, noW=1, noCL=1, channel_length=100):
    f = open('solvate_channel.inp', 'w') 
    f.write(f'tolerance 3.0\nfiletype pdb\n\n')
        
    f.write(f'structure Inner_rna.pdb\n')
    f.write(f'  number 1\n')
    f.write(f'  inside cylinder 0. 0. 0. 0. 0. 1. {(int(channel_radius)*2)+5}. {int(channel_length+10)}.\n')
    f.write(f'end structure\n')

    f.write(f'structure PDBs/water.pdb\n')
    f.write(f'  number {noW}\n')
    f.write(f'  inside cylinder 0. 0. 0. 0. 0. 1. {int(channel_radius)}. {int(channel_length)}.\n')
    f.write(f'end structure\n')

    f.write(f'structure PDBs/CL.pdb\n')
    f.write(f'  number {noCL}\n')
    f.write(f'  inside cylinder 0. 0. 0. 0. 0. 1. {int(channel_radius)}. {int(channel_length)}.\n')
    f.write(f'end structure\n')
            
    f.write('output solvate_channel_rna.pdb\n')
    f.close()
    return

def extract_lines(input_file, pattern):
    extracted_lines = []
    pattern_found = False

    with open(input_file, 'r') as f:
        for line in f:
            if pattern_found:
                extracted_lines.append(line)

            if pattern in line:
                pattern_found = True

    return extracted_lines

def write_topol (topol, noW, noCL):
    '''Writes out a topol.top file with the solvate channel times 4 including the correct itp files'''
    extracted_lines = extract_lines(topol, 'lower monolayer')
    
    itpDir   = "ITPs"
    IL_itp   = 'ITPs/ILs.itp'
    Sterols  = "ITPs//martini_v3.0_sterols_v1.0.itp"    
    ffbonded = "ITPs/martini_v3.0_ffbonded.itp"
    RNA      = 'ITPs/RNA.itp'

    with open('topol.top', 'w') as topout:
        topout.write(f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{ffbonded}"\n#include "{RNA}"\n\
#include "{itpDir}/martini_v3.0.0_phospholipids_v1.itp"\
\n#include "{IL_itp}"\n#include "{Sterols}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n\
#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n')
        topout.write(f'[ system ]\n')
        topout.write('Hexagonal phase\n')
        topout.write(f'\n')
        topout.write(f'[ molecules ]\n')
        for i in range(4):
                topout.write(f'RNA    1\n')
                for line in extracted_lines:
                    topout.write(line)
                topout.write(f'W    {noW}\n')
                topout.write(f'CL    {noCL}\n')
    return

## Importat note: Please do not proceed if you are not familiar with Packmol.

### You can learn more about Packmol here: https://m3g.github.io/packmol/userguide.shtml#tutorials

#### Note that you may need to tune tolerances, adjust lipid ratios, and make other parameter modifications

### **_Step 1_**: Create initial channel

##### write input file for pcg

In [4]:
O = open('input.str', 'w')

O.write("[Lipids List]\n")
O.write("Domain 0\n")
O.write("MC3H 0.67  0.67 0.60\n")
O.write("CHOL 0.33  0.33 0.60\n")
O.write("End\n")
O.write("\n")
O.write("[Shape Data]\n")
O.write("ShapeType Cylinder\n")
O.write("Box 10 10 10\n")
O.write("Density 2\n")
O.write("Thickness 2\n")
O.write("Radius 3\n")
O.write("End\n")
O.close()

##### Build the cylinder with PCG

In [6]:
p = subprocess.Popen(f"{PCG_path} -str input.str -Bondlength 0.2 -LLIB Martini3.LIB -defout system -function analytical_shape"
                    , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
p.wait()

==============  PCG membrane builder: from function to membrane ======================
****************** CG Membrane builder ******************   
******************  Version:  1.1 ****************** 
[Lipids List]
Domain 0
MC3H 0.67  0.67 0.60
CHOL 0.33  0.33 0.60
End

[Shape Data]
ShapeType Cylinder
vesicle will be made 
--> Generating molecule types from  input.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 145 lipid types 
--> Note: the total upper monolayer area is 251.327 and the total lower monolayer area is 125.664
 now is time to add the lipids 
 Number of the domains defined in the input file  1
***************************  we aim to generate  ********************** 
*     For domain with ID 0 
*     280  MC3H     
*     138  CHOL     
   in the upper monolayer 
*     For domain with ID 0 
*     140  MC3H     
*     69  CHOL     
   in the lower monolayer 
*************************** Num

0

##### Extract the inner leaflet for continuation

In [7]:
u = md.Universe('system.gro')

L = LeafletFinder(u, 'name NP N1 ROH', cutoff=15)

inner_res = ' '.join(f'{i}' for i in L.groups()[1].resids)
inner = u.select_atoms(f'resid {inner_res}')
inner.atoms.write('Inner.pdb')

/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/DAMM/hosts

##### solvate one channel and add neutralizing ions using packmol

In [8]:
u = md.Universe('Inner.pdb')
sel = u.select_atoms('name N1 NP ROH')
sel_lipid = u.select_atoms('name N1 NP')
R = get_radius(sel.positions[:,0], sel.positions[:,1], sel.positions[:,2]) - 4 #radius of the channel, trying to substract 2 angstorm to avoid clashing

/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "


#### Insert RNA

In [9]:
rna = md.Universe(RNA)
rna.atoms.positions = rna.atoms.positions - rna.atoms.center_of_mass()

In [10]:
u.atoms.positions = u.atoms.positions - u.atoms.center_of_mass()
sys_rna = md.Merge(rna.select_atoms('all'), u.select_atoms('all'))
sys_rna.atoms.write('Inner_rna.pdb')

/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


In [11]:
u = md.Universe('Inner_rna.pdb')

/DAMM/hosts/damm085/data1/mvalerio/my_env3.11/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [12]:
sel = u.select_atoms('name N1 NP ROH')
bb = u.select_atoms('name BB')

In [13]:
hydration = 3 #water beads per lipids, hence 12 water molecules per lipid. This is based on the paper with Nadine
noL = sel.resids.shape[0]
noCL = sel_lipid.resids.shape[0] - bb.resids.shape[0]
noW = (noL*hydration) - noCL

In [14]:
write_packmol_input(R, noW=noW, noCL=noCL)

##### Run packmol - Only use packmol for solvate and add ions

In [15]:
os.system(f'{packmol} < solvate_channel.inp')


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: solvate_channel_rna.pdb
  Reading coordinate file: Inner_rna.pdb
  Reading coordinate file: PDBs/water.pdb
  Reading coordinate file: PDBs/CL.pdb
  Number of independent structures:            3
  The structures are: 
  Structure            1 :Inner_rna.pdb(        2658  atoms)
  Structure            2 :PDBs/water.pdb(           1  atoms)
  Structure            3 :PDBs/

0

##### Dublicate the channel 4 times 

In [39]:
os.system('gmx editconf -f solvate_channel_rna.pdb -box 5.2 5.2 9.3 -o channel_solvated_combined.gro -bt triclinic -c -angles 90 90 60')
os.system('gmx genconf -f channel_solvated_combined.gro -o dublicate.gro -nbox 2 2 1 -dist 1')

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3285 atoms
No velocities found
    system size :  6.389  6.402 10.139 (nm)
    center      :  0.325  0.016  4.982 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  3.575  2.236 -0.332 (nm)
new center      :  3.900  2.252  4.650 (nm)
new box vectors :  5.200  5.200  9.300 (nm)
new box angles  :  90.00  90.00  60.00 (degrees)
new box volume  : 217.78               (nm^3)


             :-) GROMACS - gmx editconf, 2022.5-Debian_2022.5_2 (-:

Executable:   /usr/bin/gmx
Data prefix:  /usr
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/Ostrava_Workshop
Command line:
  gmx editconf -f solvate_channel_rna.pdb -box 5.2 5.2 9.3 -o channel_solvated_combined.gro -bt triclinic -c -angles 90 90 60


Back Off! I just backed up channel_solvated_combined.gro to ./#channel_solvated_combined.gro.1#

GROMACS reminds you: "I Quit My Job Blowing Leaves" (Beck)

             :-) GROMACS - gmx genconf, 2022.5-Debian_2022.5_2 (-:

Executable:   /usr/bin/gmx
Data prefix:  /usr
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/Ostrava_Workshop
Command line:
  gmx genconf -f channel_solvated_combined.gro -o dublicate.gro -nbox 2 2 1 -dist 1


Back Off! I just backed up dublicate.gro to ./#dublicate.gro.1#

GROMACS reminds you: "I Quit My Job Blowing Leaves" (Beck)



0

##### Write topol.top

In [40]:
write_topol('system.top', noW, noCL)

## Important note: Please do not proceed if you are not familiar with GROMACS.

### You can learn more about GROMACS here: http://www.mdtutorials.com/gmx/

#### Note that you may need to tune some .mdp parameters based on the GROMACS version being used.

### **_Step 2_**: Minimize System

In [41]:
p = subprocess.Popen(f"gmx grompp -f {mdp_loc_BL}/min.mdp -c dublicate.gro -p topol.top -o m.tpr -maxwarn 1"
                    , stdin=subprocess.PIPE, shell=True, universal_newlines=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
p.wait()

0

In [42]:
p = subprocess.Popen(f"gmx mdrun -v -deffnm m -ntomp 5 -ntmpi 2"
                    , stdin=subprocess.PIPE, shell=True, universal_newlines=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
p.wait()

0

In [44]:
### Create the index file
gro = 'm.gro'
u = md.Universe(gro)
lipidsagg=u.select_atoms('resname MC3H LI2H CHOL A')
solventagg=u.select_atoms('not resname MC3H LI2H CHOL A')
lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
solventagg.write("index.ndx", mode="a", name= 'Solvent')
u.atoms.write("index.ndx", mode="a", name= 'System')
